In [1]:
import math

In [2]:
class Taylor:
    def __init__(self, coef:list, order:int):
        self.order=order
        if len(coef)<(order+1):
            for i in range(order+1-len(coef)):
                coef.append(0.0)
        self.coef=[float(i) for i in coef]

    def __repr__(self):
        return 'Taylor(%s, %d)'%(self.coef, self.order)
   
    def fixshape(x, y):
        order=max(x.order,y.order)
        x1=x.coef[:]
        y1=y.coef[:]
        if x.order<order:
            for i in range(order-x.order):
                x1.append(0)
        else:
            for i in range(order-y.order):
                y1.append(0)
        return Taylor(x1,order), Taylor(y1,order), order
        
    def __add__(self, other):
        coef=[]
        self1, other1, order=self.fixshape(other)
        for i in range(order+1):
            coef.append(self1.coef[i]+other1.coef[i])
        return Taylor(coef,order)
    
    def __sub__(self, other):
        coef=[]
        self1, other1, order=self.fixshape(other)
        for i in range(order+1):
            coef.append(self1.coef[i]-other1.coef[i])
        return Taylor(coef,order)
    
    def __mul__(self, other):
        coef=[]
        self1, other1, order=Taylor.fixshape(self, other)
        coef.append(self1.coef[0]*other1.coef[0])
        for i in range(1,order+1):
            s=0
            for j in range(i+1):
                s+=self1.coef[j]*other1.coef[i-j]
            coef.append(s)
        return Taylor(coef,order)
    
    def __truediv__(self, other):
        pass

    def one(n):
        coef=[]
        coef.append(1.0)
        for i in range(n):
            coef.append(0.0)
        return Taylor(coef,n)
    
    def __eq__(self,other):
        self1, other1, order=Taylor.fixshape(self, other)
        return self1.coef==other1.coef
    
    def square(self):
        order=self.order
        coef=[]
        coef.append((self.coef[0])**2)
        for i in range(1,order+1):
            coef.append(self.squarecoef(i,self.coef))
        return Taylor(coef,order)        
    
    def squarecoef(self,i,coef):
        s=0
        iend=int((i-2+i%2)/2)
        for j in range(iend+1):
            s+=coef[j]*coef[i-j]
        s=2*s
        if i%2==0:
            s+=coef[int(i/2)]**2
        return s
    
    def sqrt(self):
        pass
    
    def power(self,n:int):
        order=self.order
        coef=self.coef
        if n<0:
            return 1/self.power(-n)
        elif n==0:
            return Taylor.one(order)
        elif n%2==0:
            if n==2:
                return self.square()
            else:
                p=int(n/2)
                return Taylor(self.power(p).square().coef,order)
        elif n==1:
            return self
        else:
            p=int((n-1)/2)
            return Taylor((self*self.power(p).square()).coef,order)                
            
    
    def exp(self):
        order=self.order
        coef=[]
        coef.append(math.exp(self.coef[0]))
        for i in range(1,order+1):
            s=0
            for j in range(i):
                s+=(i-j)*self.coef[i-j]*coef[j]
            coef.append(s/i)
        return Taylor(coef,order)
    
    def log(self):
        order=self.order
        coef=[]
        coef.append(math.log(self.coef[0]))
        for i in range(1,order+1):
            s=0
            for j in range(1,i):
                s+=(i-j)*self.coef[j]*coef[i-j]
            coef.append((self.coef[i]-s/i)/self.coef[0])
        return Taylor(coef,order)
    
    def sincos(self):
        order=self.order
        scoef=[];ccoef=[]
        scoef.append(math.sin(self.coef[0]))
        ccoef.append(math.cos(self.coef[0]))
        for i in range(1,order+1):
            s=0;c=0
            for j in range(1,i+1):
                s+=j*self.coef[j]*ccoef[i-j]
                c-=j*self.coef[j]*scoef[i-j]
            scoef.append(s/i)
            ccoef.append(c/i)
        return Taylor(scoef,order), Taylor(ccoef,order)
    
    def sin(self):
        return self.sincos()[0]
    
    def cos(self):
        return self.sincos()[1]
    
    def tan(self):
        order=self.order
        coef1=[]
        coef2=[]
        t=math.tan(self.coef[0])
        coef1.append(t)
        coef2.append(t**2)
        for i in range(1,order+1):
            coef1.append(self.tancoef(i,self.coef,coef2))
            coef2.append(self.squarecoef(i,coef1))
        return Taylor(coef1,order)
    
    def tancoef(self,i,coef,coef2):
        s=0
        for j in range(i):
            s+=(i-j)*coef[i-j]*coef2[j]
        s=self.coef[i]+s/i
        return s
    
    def diffTaylor(self):
        order=self.order
        coef=[]
        coef.append(self.coef[1])
        for i in range(2,order+1):
            coef.append(i*self.coef[i])
        coef.append(0)
        return Taylor(coef,order)
        
    def inteTaylor(self,x):
        order=self.order
        coef=[]
        coef.append(x)
        for i in range(1,order+1):
            coef.append(self.coef[i-1]/i)
        return Taylor(coef,order)
     
    def evalTaylor(self,dx):
        order=self.order
        s=self.coef[-1]
        for i in range(order-1,-1,-1):
            s=s*dx+self.coef[i]
        return s
        
    def deriv(self,n):
        s=math.factorial(n)*self.coef[n]
        return s

In [3]:
c=Taylor([2,1,2,3,4],4)
b=Taylor([1.0,2.0,3.0],2)
a=Taylor([0,1,0,0,0,0],5)
e=Taylor([1,0],1)

In [4]:
d=Taylor([0,1],17)
f=Taylor([1,1],5)
f

Taylor([1.0, 1.0, 0.0, 0.0, 0.0, 0.0], 5)

In [5]:
c+b+c

Taylor([5.0, 4.0, 7.0, 6.0, 8.0], 4)

In [6]:
c-b

Taylor([1.0, -1.0, -1.0, 3.0, 4.0], 4)

In [7]:
c*b

Taylor([2.0, 5.0, 10.0, 10.0, 16.0], 4)

In [8]:
Taylor.one(4)

Taylor([1.0, 0.0, 0.0, 0.0, 0.0], 4)

In [9]:
a.exp()

Taylor([1.0, 1.0, 0.5, 0.16666666666666666, 0.041666666666666664, 0.008333333333333333], 5)

In [10]:
(e-a).log()

Taylor([0.0, -1.0, -0.5, -0.3333333333333333, -0.25, -0.2], 5)

In [11]:
a.sincos()

(Taylor([0.0, 1.0, 0.0, -0.16666666666666666, 0.0, 0.008333333333333333], 5),
 Taylor([1.0, 0.0, -0.5, 0.0, 0.041666666666666664, 0.0], 5))

In [12]:
a.sin()

Taylor([0.0, 1.0, 0.0, -0.16666666666666666, 0.0, 0.008333333333333333], 5)

In [13]:
a.cos()

Taylor([1.0, 0.0, -0.5, 0.0, 0.041666666666666664, 0.0], 5)

In [14]:
b.square()

Taylor([1.0, 4.0, 10.0], 2)

In [15]:
a.tan()

Taylor([0.0, 1.0, 0.0, 0.3333333333333333, 0.0, 0.13333333333333333], 5)

In [16]:
b.diffTaylor()

Taylor([2.0, 6.0, 0.0], 2)

In [17]:
a.exp().diffTaylor()

Taylor([1.0, 1.0, 0.5, 0.16666666666666666, 0.041666666666666664, 0.0], 5)

In [18]:
a.exp()

Taylor([1.0, 1.0, 0.5, 0.16666666666666666, 0.041666666666666664, 0.008333333333333333], 5)

In [19]:
b.power(0)

Taylor([1.0, 0.0, 0.0], 2)

In [20]:
b.power(4)

Taylor([1.0, 8.0, 36.0], 2)

In [21]:
b.power(6)

Taylor([1.0, 12.0, 78.0], 2)

In [22]:
b.inteTaylor(1)

Taylor([1.0, 1.0, 1.0], 2)

In [23]:
a.exp().inteTaylor(0)

Taylor([0.0, 1.0, 0.5, 0.16666666666666666, 0.041666666666666664, 0.008333333333333333], 5)

In [24]:
a.exp().diffTaylor().inteTaylor(1)==a.exp()

True

In [25]:
a.exp().evalTaylor(1)

2.716666666666667

In [26]:
a.exp().evalTaylor(1)-math.exp(1)

-0.0016151617923783057

In [27]:
d.exp().evalTaylor(1)

2.718281828459045

In [28]:
d.exp().evalTaylor(1)-math.exp(1)

0.0

In [35]:
f.exp().deriv(5)==math.exp(1)

True

In [36]:
f.exp()

Taylor([2.718281828459045, 2.718281828459045, 1.3591409142295225, 0.45304697140984085, 0.11326174285246021, 0.02265234857049204], 5)